In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

In [12]:
df = pd.read_parquet('orig_data/donnee-comm-data.gouv-parquet-2023-geographie2023-produit-le2024-03-07.parquet')
df_communes = pd.read_csv('orig_data/v_commune_2023.csv')

In [13]:
tmp = df_communes[['COM', 'NCCENR']]
tmp.index = tmp.COM
tmp = tmp[['NCCENR']]

In [14]:
# Check if there are multiple keys for the same value
tmp = tmp.drop_duplicates(keep='first')

In [15]:
len(df)

3918880

In [16]:
df = df[df['CODGEO_2023'].isin(tmp.index)]

In [18]:
df['Commune'] = df['CODGEO_2023'].map(tmp.to_dict()['NCCENR'])
df['faits'] = df['faits'].fillna(0)

df['annee'] = df['annee'] + 2000

In [21]:
df = df[['Commune', 'annee', 'classe', 'faits', 'POP']]

df = df.rename(columns={
    'annee': 'Annee',
    'classe': 'Categorie',
    'faits': 'Faits',
    'POP': 'Population'
})

In [24]:
df[(df['Commune'] == 'Abancourt') & (df['Annee'] == 2023)]

,Commune,Annee,Categorie,Faits,Population
3734290,Abancourt,2023,Coups et blessures volontaires,0.0,459
3734291,Abancourt,2023,Coups et blessures volontaires intrafamiliaux,0.0,459
3734292,Abancourt,2023,Autres coups et blessures volontaires,0.0,459
3734293,Abancourt,2023,Violences sexuelles,0.0,459
3734294,Abancourt,2023,Vols avec armes,0.0,459
3734295,Abancourt,2023,Vols violents sans arme,0.0,459
3734296,Abancourt,2023,Vols sans violence contre des personnes,0.0,459
3734297,Abancourt,2023,Cambriolages de logement,0.0,459
3734298,Abancourt,2023,Vols de véhicules,0.0,459
3734299,Abancourt,2023,Vols dans les véhicules,0.0,459


In [22]:
for year in df['Annee'].unique():
    df[df['Annee'] == year].to_csv(f'data/data_{year}.csv', index=False)

In [4]:
df = df[df['annee'] == 23]

In [5]:
# Fill the NA by 0 for the columns 'faits' and 'tauxpourmille'
df['faits'] = df['faits'].fillna(0)
df['tauxpourmille'] = df['tauxpourmille'].fillna(0)

In [16]:
choice_classes = df['classe'].unique()#['Coups et blessures volontaires', 'Autres coups et blessures volontaires']
choice_classes = [
    'Coups et blessures volontaires',
    'Coups et blessures volontaires intrafamiliaux',
    'Autres coups et blessures volontaires'
]
tmp = df[df['classe'].isin(choice_classes)]
#tmp = tmp[tmp['POP'] >= 22500]

In [17]:
values = tmp.groupby('CODGEO_2023')['faits'].sum() / tmp.groupby('CODGEO_2023')['POP'].mean()

In [18]:
(tmp.groupby('CODGEO_2023')['faits'].sum() / tmp.groupby('CODGEO_2023')['POP'].mean()).sort_values(ascending=False)

CODGEO_2023
95527    0.159240
19262    0.083333
75101    0.061562
62447    0.049505
62765    0.042835
           ...   
55050         NaN
55139         NaN
55189         NaN
55239         NaN
55307         NaN
Length: 34990, dtype: float64

In [19]:
tmp_list = values.sort_values(ascending=False).head(20)
tmp_list

CODGEO_2023
95527    0.159240
19262    0.083333
75101    0.061562
62447    0.049505
62765    0.042835
69299    0.042075
25112    0.041384
97120    0.041281
13201    0.039304
14220    0.039271
59036    0.037882
60589    0.036722
33402    0.036711
75108    0.035931
59136    0.035503
59606    0.035217
16292    0.035192
62826    0.034180
62498    0.034092
97105    0.033950
dtype: float64

In [20]:
cntr = 1

for i in tmp_list.index:
    name = df_communes[df_communes['COM'] == i]['NCCENR'].values[0]

    nbr_crimes = df[df['CODGEO_2023'] == i]['faits'].sum()
    pop = df[df['CODGEO_2023'] == i]['POP'].mean()

    taux = nbr_crimes / pop * 1000

    print(f"{cntr:>2}. {name:>30} - {taux:>6.1f} - {nbr_crimes:>5.0f} - {pop:>7.0f}")

    cntr += 1

 1.               Roissy-en-France - 1691.7 -  4632 -    2738
 2.           Soudaine-Lavinadière -  118.1 -    17 -     144
 3.       Paris 1er Arrondissement -  646.1 - 10286 -   15919
 4.                         Hesdin -  107.6 -   239 -    2222
 5.                     Saint-Omer -  108.5 -  1590 -   14661
 6.             Colombier-Saugnieu -  232.5 -   641 -    2757
 7.                     Chalezeule -  133.2 -   177 -    1329
 8.                 Pointe-à-Pitre -  137.7 -  1995 -   14486
 9.   Marseille 1er Arrondissement -  341.4 - 13464 -   39436
10.                      Deauville -  154.3 -   550 -    3565
11.              Avesnes-sur-Helpe -   76.3 -   314 -    4118
12.                  Saint-Maximin -  121.0 -   356 -    2941
13.           Sainte-Foy-la-Grande -   80.7 -   211 -    2615
14.        Paris 8e Arrondissement -  370.1 - 12998 -   35123
15.               Cateau-Cambrésis -   83.7 -   580 -    6929
16.                   Valenciennes -  100.2 -  4307 -   42991
17.     

In [102]:
aaa = tmp.groupby('CODGEO_2023')['faits'].sum()
aaa[aaa.index == '59350']

CODGEO_2023
95527    4632.0
Name: faits, dtype: float64

In [103]:
df[df['CODGEO_2023'] == '95527']

,CODGEO_2023,annee,classe,unité.de.compte,valeur.publiée,faits,tauxpourmille,complementinfoval,complementinfotaux,POP,millPOP,LOG,millLOG
3916500,95527,23,Coups et blessures volontaires,victime,diff,218.0,79.620161,NaN,NaN,2738,21,1485.134858,20
3916501,95527,23,Coups et blessures volontaires intrafamiliaux,victime,diff,45.0,16.435354,NaN,NaN,2738,21,1485.134858,20
3916502,95527,23,Autres coups et blessures volontaires,victime,diff,173.0,63.184806,NaN,NaN,2738,21,1485.134858,20
3916503,95527,23,Violences sexuelles,victime,diff,57.0,20.818115,NaN,NaN,2738,21,1485.134858,20
3916504,95527,23,Vols avec armes,infraction,ndiff,0.0,0.000000,1.448718,0.140130,2738,21,1485.134858,20
3916505,95527,23,Vols violents sans arme,infraction,diff,32.0,11.687363,NaN,NaN,2738,21,1485.134858,20
3916506,95527,23,Vols sans violence contre des personnes,victime entendue,diff,3087.0,1127.465303,NaN,NaN,2738,21,1485.134858,20
3916507,95527,23,Cambriolages de logement,infraction,ndiff,0.0,0.000000,2.604167,6.246055,2738,21,1485.134858,20
3916508,95527,23,Vols de véhicules,véhicule,diff,73.0,26.661797,NaN,NaN,2738,21,1485.134858,20
3916509,95527,23,Vols dans les véhicules,véhicule,diff,238.0,86.924763,NaN,NaN,2738,21,1485.134858,20


In [92]:
for i in df['classe'].unique():

    print(i)

Coups et blessures volontaires
Coups et blessures volontaires intrafamiliaux
Autres coups et blessures volontaires
Violences sexuelles
Vols avec armes
Vols violents sans arme
Vols sans violence contre des personnes
Cambriolages de logement
Vols de véhicules
Vols dans les véhicules
Vols d'accessoires sur véhicules
Destructions et dégradations volontaires
Trafic de stupéfiants
Usage de stupéfiants


In [123]:
df[df['CODGEO_2023'] == '95527'][['classe', 'faits', 'POP']]

,classe,faits,POP
3916500,Coups et blessures volontaires,218.0,2738
3916501,Coups et blessures volontaires intrafamiliaux,45.0,2738
3916502,Autres coups et blessures volontaires,173.0,2738
3916503,Violences sexuelles,57.0,2738
3916504,Vols avec armes,0.0,2738
3916505,Vols violents sans arme,32.0,2738
3916506,Vols sans violence contre des personnes,3087.0,2738
3916507,Cambriolages de logement,0.0,2738
3916508,Vols de véhicules,73.0,2738
3916509,Vols dans les véhicules,238.0,2738


In [27]:
tmp = df_communes[['COM', 'NCCENR']]
tmp.index = tmp.COM
tmp = tmp[['NCCENR']]

In [30]:
df['Commune'] = df['CODGEO_2023'].map(tmp.to_dict()['NCCENR'])

In [32]:
df[['Commune', 'annee', 'classe', 'faits', 'POP']]

,Commune,annee,classe,faits,POP
3429020,Abergement-Clémenciat,23,Coups et blessures volontaires,0.0,832
3429021,Abergement-Clémenciat,23,Coups et blessures volontaires intrafamiliaux,0.0,832
3429022,Abergement-Clémenciat,23,Autres coups et blessures volontaires,0.0,832
3429023,Abergement-Clémenciat,23,Violences sexuelles,0.0,832
3429024,Abergement-Clémenciat,23,Vols avec armes,0.0,832
...,...,...,...,...,...
3918875,Tsingoni,23,Vols dans les véhicules,19.0,13934
3918876,Tsingoni,23,Vols d'accessoires sur véhicules,0.0,13934
3918877,Tsingoni,23,Destructions et dégradations volontaires,91.0,13934
3918878,Tsingoni,23,Trafic de stupéfiants,0.0,13934
